In [1]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.common.exceptions import WebDriverException, NoSuchElementException, TimeoutException
import logging
from bs4 import BeautifulSoup
import time
import pandas as pd


class WebScraper_NewsGoogle:
    def __init__(self, keyword):
        self.keyword = keyword
        self.driver = None
        self.news_data = []
        
        self.setup_driver()
        self.search_for_keyword()
        self.click_on_news()

    def setup_driver(self):
        
        chrome_options = webdriver.ChromeOptions()
        chrome_options.add_argument("--disable-notifications")
        #chrome_options.add_argument("--headless")  # Add this line to run in headless mode
        service = Service(executable_path='C:/Users/limam/Documents/Programação/Python/Projetos/Web Scraping/chromedriver.exe')
        # Pass the service object to the webdriver
        self.driver = webdriver.Chrome(options=chrome_options, service=service) 
        self.driver.get("http://www.google.com/")
        
        try:
            WebDriverWait(self.driver, 5).until(EC.presence_of_element_located((By.NAME, 'q')))
        except WebDriverException:
            print("Tweets did not appear! Proceeding after timeout")

    def search_for_keyword(self):
        
        search_box = self.driver.find_element(By.NAME, 'q')
        search_box.send_keys(self.keyword)
        search_box.submit()

    def click_on_news(self):
        try:
            WebDriverWait(self.driver, 5).until(EC.presence_of_element_located((By.LINK_TEXT, "Notícias")))
            element = self.driver.find_element(By.LINK_TEXT, "Notícias")
            element.click()
                       
        except NoSuchElementException as e:
            
            WebDriverWait(self.driver, 5).until(EC.presence_of_element_located((By.LINK_TEXT, "News")))
            element = self.driver.find_element(By.LINK_TEXT, "News")
            element.click() 

    def scrape_data(self):
        News_data = []
        try:
            while True:
                try:
                    # Tente encontrar os resultados de notícias usando o seletor CSS
                    try:
                        news_results = self.driver.find_elements(By.CSS_SELECTOR, 'div#rso > div > div > div > div')
                        for news_div in news_results:   
                            try:
                                news_item = {}

                                try:
                                    news_link = news_div.find_element(By.TAG_NAME, 'a').get_attribute('href')
                                    news_item["Link"] = news_link
                                except NoSuchElementException as e:
                                    print("Link not found:", e)
                                    logging.error("Link not found: " + str(e))
                                    news_item["Link"] = None
                                    continue  # Continue para o próximo item de notícia

                                # Extrair o domínio
                                domain_elements = news_div.find_elements(By.CSS_SELECTOR, 'a > div > div > div')

                                if len(domain_elements) >= 2:
                                    domain = domain_elements[1].text
                                    news_item["Domain"] = domain
                                else:
                                    print("Domain not found")
                                    news_item["Domain"] = None

                                # Extrair o título
                                if len(domain_elements) >= 3:
                                    title = domain_elements[2].text
                                    news_item["Title"] = title
                                else:
                                    print("Title not found")
                                    news_item["Title"] = None

                                # Extrair a descrição
                                if len(domain_elements) >= 4:
                                    description = domain_elements[3].text
                                    news_item["Description"] = description
                                else:
                                    print("Description not found")
                                    news_item["Description"] = None

                                # Extrair a data
                                if len(domain_elements) >= 5:
                                    date = domain_elements[4].text
                                    news_item["Date"] = date
                                else:
                                    try:
                                        Date = news_div.find_element(By.CSS_SELECTOR, '#rso > div > div > div.SoaBEf.R24aHf > div > div > a > div > div.iRPxbe > div.OSrXXb.rbYSKb.LfVVr')
                                        news_item["Date"] = Date
                                    except NoSuchElementException as e:
                                        #print("Date not found")
                                        news_item["Date"] = None
                                        continue  # Continue para o próximo item de notícia
                                   


                                print("Link:", news_item["Link"])
                                print("Domain:", news_item["Domain"])
                                print("Title:", news_item["Title"])
                                print("Description:", news_item["Description"])
                                print("Date:", news_item["Date"])
                                print("-" * 50 + "\n\n" + "-" * 50)

                                try:
                                    News_data.append(news_item)

                                except Exception as e:
                                    print("Error appending news_item to News_data:", e)
                                    logging.error("Error appending news_item to News_data: " + str(e))


                            except NoSuchElementException as e:
                                print("Incomplete news item:", e)
                                logging.error("Incomplete news item: " + str(e))

                                try:                                
                                    WebDriverWait(self.driver, 5).until(EC.presence_of_element_located((By.ID, "pnnext")))
                                    elemento_proxima = self.driver.find_element(By.ID, "pnnext")
                                    #time.sleep(2)
                                    elemento_proxima.click()

                                except NoSuchElementException:
                                    # Tratamento de erro: O botão "Próxima" não foi encontrado, indicando o fim das páginas
                                    print("Não há mais páginas disponíveis.")


                    except NoSuchElementException as e:
                        print("News results not found:", e)
                        logging.error("News results not found: " + str(e))
                        # Sair do loop se os resultados de notícias não forem encontrados

                except NoSuchElementException:
                    # Tratamento de erro: O elemento "Próxima" não foi encontrado, ou seja, não há mais páginas
                    print("Não há mais páginas disponíveis.")

                # Encontre o elemento "Próxima" (ou equivalente) pelo ID ou outro seletor
                try:
                    elemento_proxima = self.driver.find_element(By.ID, "pnnext")
                    elemento_proxima.click()

                except NoSuchElementException:
                    # Tratamento de erro: O botão "Próxima" não foi encontrado, indicando o fim das páginas
                    print("Não há mais páginas disponíveis.")
                    break

        except TimeoutException as e:
            print(f"Timeout Exception: {e}")
            logging.error("Timeout Exception: " + str(e))

        except Exception as e:
            print(f"An error occurred: {e}")
            logging.error("An error occurred: " + str(e))

        #finally:
            # Feche o navegador após a conclusão
            #driver.quit()

        return News_data

    def close_driver(self):
        if self.driver:
            self.driver.quit()
            print("Realizada")

if __name__ == "__main__":
    print("Pronto Para começar!")

Pronto Para começar!


In [2]:
data_list_10 = []
data_list_15 = []
data_list_25 = []

In [3]:
keyword = 'Debentures'

In [5]:
Google_news = WebScraper_NewsGoogle(keyword)
News = Google_news.scrape_data()

data_list_10.extend(News[:10])
data_list_15.extend(News[:15])
data_list_25.extend(News[:25])

Google_news.close_driver()

Link: https://www.cnnbrasil.com.br/economia/macroeconomia/governo-lula-sanciona-lei-que-cria-debentures-para-financiar-projetos-de-infraestrutura/
Domain: CNN Brasil
Title: Governo Lula sanciona lei que cria debêntures para financiar projetos de infraestrutura
Description: O presidente Lula sancionou nesta quarta-feira (10) a lei que cria debêntures para financiar projetos em infraestrutura.
Date: 1 dia atrás
--------------------------------------------------

--------------------------------------------------
Link: https://einvestidor.estadao.com.br/investimentos/americanas-amer3-fraude-um-ano-debentures/
Domain: E-Investidor
Title: O que o estrago da Americanas (AMER3) causou no mercado de debêntures 1 ano depois – Investimentos – Estadão E-Investidor – As principais notícias do mercado financeiro
Description: O escândalo com o rombo contábil de R$ 25 bilhões da Americanas (AMER3) completa um ano nesta quinta-feira (11). Naquela noite, o ex-CEO...
Date: 16 horas atrás
---------------